# **Samuel Vasco Vasco González**  Cc. 1152223665

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
#lybraries
%pylab inline
import pandas as pd
from mpl_toolkits import mplot3d
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error
from tabulate import tabulate

Populating the interactive namespace from numpy and matplotlib


ModuleNotFoundError: No module named 'tabulate'

In [ ]:
data=pd.read_csv("uelConsumption.csv")
data

Este data set contiene el índice de consumo de combustible de modelos específicos y un estimado de las emisiones de Dióxido de carbono para vehículos ligeros nuevos para la venta al por menor en Canadá.

In [ ]:
data_extract=data[['CO2EMISSIONS','ENGINESIZE','CYLINDERS','FUELCONSUMPTION_CITY','FUELCONSUMPTION_HWY','FUELCONSUMPTION_COMB']]
data_extract

Una vez seleccionadas estas características, implementaremos un modelo de regresión lineal que busca predecir las emisiones de CO2 (CO2EMISSIONS) a partir de características o predictores como el tamaño del motor (ENGINESIZE), el número de cilindros (CYLINDERS), consumo de gasolina en la ciudad (FUELCONSUMPTION_CITY), consumo de gasolina en carretera (FUELCONSUMPTION_HWY) y FUELCONSUMPTION_COMB.

# Visualize the data in 2D


In [ ]:
Enginesize=data_extract["ENGINESIZE"].values
Cylinders=data_extract["CYLINDERS"].values
Co2emissions=data_extract["CO2EMISSIONS"].values

fig,axs=plt.subplots(1,2)
fig=figsize(15,8)

axs[0].plot(Enginesize,Co2emissions,"bo")
axs[0].set_xlabel("Engine Size")
axs[0].set_ylabel("Co2 Emissions")
axs[0].set_title("CO2 EMISSIONS vs ENGINE SIZE ")
axs[1].plot(Cylinders,Co2emissions,"bo")
axs[1].set_xlabel("Cylinders")
axs[1].set_ylabel("Co2 Emissions")
axs[1].set_title("CO2 EMISSIONS vs CYLINDERS")

Podemos notar que en ambas gráficas la dispersión de los puntos parecen tener un comportamiento lineal.

# Plot 3D

In [ ]:
fig = plt.figure(figsize=(10,8))
ax = plt.axes(projection ='3d')
 
x = Enginesize
y = Cylinders
z = Co2emissions
ax.plot(x, y, z,"bo") 

ax.set_xlabel('ENGINE SIZE')
ax.set_ylabel('CYLINDERS')
ax.set_zlabel('CO2 EMISSIONS') 
ax.set_title('ENGINE SIZE and CYLINDERS vs CO2 EMISSIONS')
plt.show()

Es posible apreciar que existe un plano al cual se pueden ajustar los puntos.

# Apply feature scaling.

##  It is a step of Data Pre Processing that is applied to independent variables or features of data. It basically helps to normalize the data within a particular range. Sometimes, it also helps in **speeding up the calculations in an algorithm.**

We use Standarscaler, it replace the values with their Z score. 
**Z score** is the number of standard desviations that the values is far away from the mean.

$$Z=\dfrac{x-\mu}{\sigma}$$

In [ ]:
# Initialise the Scaler
scaler = StandardScaler()
 
# To scale data
df=scaler.fit_transform(data_extract)
df

Now each column of data is scaled to mean $\mu=0$ and standard desviation $\sigma=1$. This values are equivalents to the initials values. The distribution of data stay equal, it is invariant under scale transformation.

In [ ]:
#Data without Scaler
#data_scaler=data_extract

#Data with Scaler
data_scaler=pd.DataFrame(df,columns=['CO2EMISSIONS','ENGINESIZE','CYLINDERS','FUELCONSUMPTION_CITY','FUELCONSUMPTION_HWY','FUELCONSUMPTION_COMB'])
data_scaler

# Split the data into train and test using scikit-learn

In [ ]:
# División de los datos en train y test
# ==============================================================================
#Features 
X = data_scaler[['ENGINESIZE','CYLINDERS','FUELCONSUMPTION_CITY','FUELCONSUMPTION_HWY','FUELCONSUMPTION_COMB']]
#Target variable
y = data_scaler[['CO2EMISSIONS']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                  random_state=43,shuffle=True)

División de los datos en Train y Test con una distribución del 80% y 20% del total de datos respectivamente.

# Train the Linear Model, Predictions from model and Print the Metrics $R^2$, MAE and MAPE. 

In [ ]:
#Modelo
#==============================================================================
linear_regression = LinearRegression()

#Entrenamiento del modelo
# ==============================================================================
linear_regression.fit(X_train,y_train)

# Predicciones test
# ==============================================================================
predicciones = linear_regression.predict(X=X_test)
predicciones = predicciones.flatten()

#Metric R2
# ==============================================================================
R2_Train=linear_regression.score(X_train,y_train)
R2_Test=linear_regression.score(X_test,y_test)

print("The R2 for Train set is: {:.4}".format(R2_Train))
print("The R2 for Test set is: {:.4} \n".format(R2_Test))

#Metric MAE
# ==============================================================================
MAE=mean_absolute_error(y_test,predicciones)

print("Metric MAE is: {:.4} \n".format(MAE))

#Metric MAPE
# ==============================================================================
MAPE=mean_absolute_percentage_error(y_test,predicciones)

print("Metric MAPE is: {:.4} \n".format(MAPE))

El modelo tiene un Score $R^2=0.87$ para Test lo cual resulta bastante bien. 

In [ ]:
#Parameters from Linear Model
# ==============================================================================
Coef=linear_regression.coef_.flatten()
Interc=linear_regression.intercept_.flatten()

print("Vector of Coeficients from Linear Regression Model: \n {} \n".format(Coef))
print("Intercept from Linear Regression Model: \n {}".format(Interc))

# Predict CO2 emission of the 10 randomly chosen cars, compare with the true values

In [ ]:

y_test_array=y_test.values.flatten()

#choose 10 randomly cars
random=np.random.randint(len(y_test_array), size=10)
random_True=[]
random_pred=[]
for i in random:
  random_True.append(y_test_array[i])
  random_pred.append(predicciones[i])

dict={"CO2 emission True":random_True, "CO2 emission Predict":random_pred}

print(tabulate(dict, headers='keys', tablefmt='fancy_grid')) 
  

Notamos que hay valores bastante cercanos entre el predicho y el real, mientras que hay otros bastante lejanos entre el predicho y el real. 

Interpretando el MAE, tenemos que en promedio los datos predichos tienen un error de masomenos 0.2691, es decir, los valores predichos deben interpretarse así: $CO2emission_{pred}\pm MAE = valor$ $predicho \pm 0.2691$. 

Y como en este caso los valores estaban escalados como el número de desviaciones típicas que un valor dado toma con respecto a la media, entonces tenemos un error en el valor predicho bastante significativo.

Esto nos lo confirma el MAPE, pues nos está indicando un error porcentual en la predicción de cada emsion de CO2 del $74.2$%, un error significativamente grande.


In [ ]:
#Data without Scaler
data_scaler=data_extract

# División de los datos en train y test
# ==============================================================================
#Features 
X = data_scaler[['ENGINESIZE','CYLINDERS','FUELCONSUMPTION_CITY','FUELCONSUMPTION_HWY','FUELCONSUMPTION_COMB']]
#Target variable
y = data_scaler[['CO2EMISSIONS']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                  random_state=43,shuffle=True)

#Modelo
#==============================================================================
linear_regression = LinearRegression()

#Entrenamiento del modelo
# ==============================================================================
linear_regression.fit(X_train,y_train)

# Predicciones test
# ==============================================================================
predicciones = linear_regression.predict(X=X_test)
predicciones = predicciones.flatten()

#Metric R2
# ==============================================================================
R2_Train=linear_regression.score(X_train,y_train)
R2_Test=linear_regression.score(X_test,y_test)

print("The R2 for Train set is: {:.4}".format(R2_Train))
print("The R2 for Test set is: {:.4} \n".format(R2_Test))

#Metric MAE
# ==============================================================================
MAE=mean_absolute_error(y_test,predicciones)

print("Metric MAE is: {:.4} \n".format(MAE))

#Metric MAPE
# ==============================================================================
MAPE=mean_absolute_percentage_error(y_test,predicciones)

print("Metric MAPE is: {:.4} \n".format(MAPE))

#Parameters from Linear Model
# ==============================================================================
Coef=linear_regression.coef_.flatten()
Interc=linear_regression.intercept_.flatten()

print("Vector of Coeficients from Linear Regression Model: \n {} \n".format(Coef))
print("Intercept from Linear Regression Model: \n {}".format(Interc))

y_test_array=y_test.values.flatten()

#choose 10 randomly cars
random=np.random.randint(len(y_test_array), size=10)
random_True=[]
random_pred=[]
for i in random:
  random_True.append(y_test_array[i])
  random_pred.append(predicciones[i])

dict={"CO2 emission True":random_True, "CO2 emission Predict":random_pred}

print(tabulate(dict, headers='keys', tablefmt='fancy_grid')) 

Para los datos sin Escalar, obtenemos resultados más acordes con las métricas obtenidas. Pues en la predicción de cada valor tenemos un error así: 

$$valor_{predicho} \pm 17.05$$

Cabe recalcar qeu MAE está en la misma escala que la variable objetivo.

E interpretando MAPE, en cada valor predicho tenemos un $6.274$% de error respecto a los valores reales.

Todo esto resulta genial, nuestro modelo sirve para predecir CO2 emissions. Y ahora sí, podemos decir que todo el modelo tiene un rendimiento muy bueno, el cual es reportado por $R^2$ $Score=87$%.

###**En Conclusión**

Para estos datos es más conveniente no usar un escalador de los datos, pues aunque estamos en una regresión lineal y usar esto supone un mejor rendimiento del algorimo, en este caso nos lleva a pérdidas de interpretabilidad. 